In [4]:
%cd practicas/pr403

/media/notebooks/practicas/pr403


/usr/local/lib/python3.10/dist-packages/IPython/core/magics/osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [2]:
!hdfs dfs -mkdir /pr403

mkdir: `/pr403': File exists


In [3]:
!hdfs dfs -put ./logs.log /pr403

put: `/pr403/logs.log': File exists


# Ejercicio 1

In [4]:
%%writefile mapper1.py
#!/usr/bin/env python3

import sys

for line in sys.stdin:
    line = line.strip().split()
    print(f"{line[8]}, {1}")


Overwriting mapper1.py


In [5]:
%%writefile reducer1.py
#!/usr/bin/env python3


import sys

dic = {}

for line in sys.stdin:
    line = line.strip().split(", ")
    num = int(line[0])
    count = int(line[1])

    if num in dic:
        dic[num] = dic[num] + count
    else:
        dic[num] = count

for num, count in dic.items():
    print(f"{num}: {count}")


Overwriting reducer1.py


In [6]:
!hdfs dfs -rm -r /pr403/salida1

Deleted /pr403/salida1


In [7]:
!hadoop jar \
/usr/local/hadoop/share/hadoop/tools/lib/hadoop-streaming-3.4.0.jar \
-file mapper1.py \
-file reducer1.py \
-mapper mapper1.py \
-reducer reducer1.py \
-input /pr403/logs.log \
-output /pr403/salida1

2025-12-03 16:54:58,794 WARN streaming.StreamJob: -file option is deprecated, please use generic option -files instead.
packageJobJar: [mapper1.py, reducer1.py, /tmp/hadoop-unjar4699871641485395965/] [] /tmp/streamjob7467766486674358150.jar tmpDir=null
2025-12-03 16:54:59,389 INFO client.DefaultNoHARMFailoverProxyProvider: Connecting to ResourceManager at yarnmanager/172.20.0.4:8032
2025-12-03 16:54:59,495 INFO client.DefaultNoHARMFailoverProxyProvider: Connecting to ResourceManager at yarnmanager/172.20.0.4:8032
2025-12-03 16:54:59,699 INFO mapreduce.JobResourceUploader: Disabling Erasure Coding for path: /tmp/hadoop-yarn/staging/root/.staging/job_1764780814862_0001
2025-12-03 16:55:00,837 INFO mapred.FileInputFormat: Total input files to process : 1
2025-12-03 16:55:00,923 INFO mapreduce.JobSubmitter: number of splits:2
2025-12-03 16:55:01,041 INFO mapreduce.JobSubmitter: Submitting tokens for job: job_1764780814862_0001
2025-12-03 16:55:01,041 INFO mapreduce.JobSubmitter: Executing 

In [8]:
!hdfs dfs -cat /pr403/salida1/part-00000

200: 15	
303: 15	
304: 17	
403: 8	
404: 12	
500: 15	
502: 18	


In [9]:
!cat logs.log | python3 mapper1.py | sort | python3 reducer1.py 

200: 15
303: 15
304: 17
403: 8
404: 12
500: 15
502: 18


# Tráfico Total por IP

In [29]:
%%writefile mapper2.py
#!/usr/bin/env python3

import sys

for line in sys.stdin:
    line = line.strip().split()
    ip = line[0]
    if line[9] == "-":
         bytes = 0
    else:
        bytes = int(line[9])

print(f"{ip}, {bytes}")

Overwriting mapper2.py


In [30]:
%%writefile reducer2.py
#!/usr/bin/env python3

import sys

dic = {}

for line in sys.stdin:
    line = line.strip().split(",")
    ip = line[0]
    bytes = int(line[1])

    if ip in dic:
        dic[ip] = dic[ip] + bytes
    else:
        dic[ip] = bytes

for ip, bytes in dic.items(): 
    print(f"{ip}: {bytes} bytes")

Overwriting reducer2.py


In [31]:
!hdfs dfs -rm -r /pr403/salida2

Deleted /pr403/salida2


In [32]:
!hadoop jar \
/usr/local/hadoop/share/hadoop/tools/lib/hadoop-streaming-3.4.0.jar \
-file mapper2.py \
-file reducer2.py \
-mapper mapper2.py \
-reducer reducer2.py \
-input /pr403/logs.log \
-output /pr403/salida2

2025-12-03 17:10:45,963 WARN streaming.StreamJob: -file option is deprecated, please use generic option -files instead.
packageJobJar: [mapper2.py, reducer2.py, /tmp/hadoop-unjar8038086215335020674/] [] /tmp/streamjob3828672126128387049.jar tmpDir=null
2025-12-03 17:10:46,625 INFO client.DefaultNoHARMFailoverProxyProvider: Connecting to ResourceManager at yarnmanager/172.20.0.4:8032
2025-12-03 17:10:46,746 INFO client.DefaultNoHARMFailoverProxyProvider: Connecting to ResourceManager at yarnmanager/172.20.0.4:8032
2025-12-03 17:10:46,916 INFO mapreduce.JobResourceUploader: Disabling Erasure Coding for path: /tmp/hadoop-yarn/staging/root/.staging/job_1764780814862_0006
2025-12-03 17:10:47,180 INFO mapred.FileInputFormat: Total input files to process : 1
2025-12-03 17:10:47,235 INFO mapreduce.JobSubmitter: number of splits:2
2025-12-03 17:10:47,314 INFO mapreduce.JobSubmitter: Submitting tokens for job: job_1764780814862_0006
2025-12-03 17:10:47,314 INFO mapreduce.JobSubmitter: Executing 

In [33]:
!hdfs dfs -cat /pr403/salida2/part-00000

250.68.88.150: 4972 bytes	
46.141.41.90: 4981 bytes	


# 2. Análisis de comportamiento
## URLS más pupulares

In [53]:
%%writefile mapper3.py
#!/usr/bin/env python3

import sys

for line in sys.stdin:
    line = line.strip().split()
    url = line[6]

    print(f"{url}, {1}")

Overwriting mapper3.py


In [54]:
%%writefile reducer3.py
#!/usr/bin/env python3

import sys

dic = {}

for line in sys.stdin:
    line = line.strip().split()
    url = line[0]
    count = int(line[1])

    if url in dic:
        dic[url] = dic[url] + count
    else:
        dic[url] = count

for url, count in dic.items():
    print(f"{url}, {count}")

Overwriting reducer3.py


In [55]:
!hdfs dfs -rm -r /pr403/salida3

Deleted /pr403/salida3


In [56]:
!hadoop jar \
/usr/local/hadoop/share/hadoop/tools/lib/hadoop-streaming-3.4.0.jar \
-file mapper3.py \
-file reducer3.py \
-mapper mapper3.py \
-reducer reducer3.py \
-input /pr403/logs.log \
-output /pr403/salida3

2025-12-03 17:27:08,021 WARN streaming.StreamJob: -file option is deprecated, please use generic option -files instead.
packageJobJar: [mapper3.py, reducer3.py, /tmp/hadoop-unjar4582899542856014381/] [] /tmp/streamjob9214678586365403708.jar tmpDir=null
2025-12-03 17:27:08,591 INFO client.DefaultNoHARMFailoverProxyProvider: Connecting to ResourceManager at yarnmanager/172.20.0.4:8032
2025-12-03 17:27:08,707 INFO client.DefaultNoHARMFailoverProxyProvider: Connecting to ResourceManager at yarnmanager/172.20.0.4:8032
2025-12-03 17:27:08,849 INFO mapreduce.JobResourceUploader: Disabling Erasure Coding for path: /tmp/hadoop-yarn/staging/root/.staging/job_1764780814862_0011
2025-12-03 17:27:09,167 INFO mapred.FileInputFormat: Total input files to process : 1
2025-12-03 17:27:09,258 INFO mapreduce.JobSubmitter: number of splits:2
2025-12-03 17:27:09,350 INFO mapreduce.JobSubmitter: Submitting tokens for job: job_1764780814862_0011
2025-12-03 17:27:09,350 INFO mapreduce.JobSubmitter: Executing 

In [57]:
!hdfs dfs -cat /pr403/salida3/part-00000

/usr,, 23	
/usr/admin,, 17	
/usr/admin/developer,, 18	
/usr/login,, 21	
/usr/register,, 21	


## Distribución por método HTTP

In [6]:
%%writefile mapper4.py
#!/usr/bin/env python3

import sys

for line in sys.stdin:
    line = line.strip().split()
    http = line[5]
    print(f"{http}, {1}")

Overwriting mapper4.py


In [11]:
%%writefile reducer4.py
#!/usr/bin/env python3

import sys

dic = {}

for line in sys.stdin:
    line = line.strip().split(",")
    http = line[0]
    count = int(line[1])

    if http in dic:
        dic[http] = dic[http] + count
    else:
        dic[http] = count

for http, count in dic.items():
    print(f"{http}, {count}")

Overwriting reducer4.py


In [12]:
!hdfs dfs -rm -r /pr403/salida4

Deleted /pr403/salida4


In [13]:
!hadoop jar \
/usr/local/hadoop/share/hadoop/tools/lib/hadoop-streaming-3.4.0.jar \
-file mapper4.py \
-file reducer4.py \
-mapper mapper4.py \
-reducer reducer4.py \
-input /pr403/logs.log \
-output /pr403/salida4

2025-12-06 17:55:37,996 WARN streaming.StreamJob: -file option is deprecated, please use generic option -files instead.
packageJobJar: [mapper4.py, reducer4.py, /tmp/hadoop-unjar5139796246872456713/] [] /tmp/streamjob5726218660135801296.jar tmpDir=null
2025-12-06 17:55:38,654 INFO client.DefaultNoHARMFailoverProxyProvider: Connecting to ResourceManager at yarnmanager/172.20.0.3:8032
2025-12-06 17:55:38,790 INFO client.DefaultNoHARMFailoverProxyProvider: Connecting to ResourceManager at yarnmanager/172.20.0.3:8032
2025-12-06 17:55:38,969 INFO mapreduce.JobResourceUploader: Disabling Erasure Coding for path: /tmp/hadoop-yarn/staging/root/.staging/job_1765043067955_0002
2025-12-06 17:55:39,289 INFO mapred.FileInputFormat: Total input files to process : 1
2025-12-06 17:55:39,355 INFO mapreduce.JobSubmitter: number of splits:2
2025-12-06 17:55:39,442 INFO mapreduce.JobSubmitter: Submitting tokens for job: job_1765043067955_0002
2025-12-06 17:55:39,442 INFO mapreduce.JobSubmitter: Executing 

In [14]:
!hdfs dfs -cat /pr403/salida4/part-00000

"DELETE, 23	
"GET, 25	
"POST, 24	
"PUT, 28	


## Analisis de navegadores

In [37]:
%%writefile mapper5.py
#!/usr/bin/env python3

import sys

for line in sys.stdin:
    line = line.strip().split()
    navegador = line[11]
    print(f"{navegador}, {1}")

Overwriting mapper5.py


In [48]:
%%writefile reducer5.py
#!/usr/bin/env python3

import sys

dic = {}

for line in sys.stdin:
    line = line.strip().split(",")
    line = line[0]
    if line in dic:
        dic[line] = dic[line] + 1
    else:
        dic[line] = 1

for line, count in dic.items():
    print(f"{line}, {count}")

Overwriting reducer5.py


In [39]:
!hdfs dfs -rm -r /pr403/salida5

Deleted /pr403/salida5


In [40]:
!hadoop jar \
/usr/local/hadoop/share/hadoop/tools/lib/hadoop-streaming-3.4.0.jar \
-file mapper5.py \
-file reducer5.py \
-mapper mapper5.py \
-reducer reducer5.py \
-input /pr403/logs.log \
-output /pr403/salida5

2025-12-06 18:08:21,357 WARN streaming.StreamJob: -file option is deprecated, please use generic option -files instead.
packageJobJar: [mapper5.py, reducer5.py, /tmp/hadoop-unjar6973581599727951154/] [] /tmp/streamjob2358296235929841562.jar tmpDir=null
2025-12-06 18:08:21,876 INFO client.DefaultNoHARMFailoverProxyProvider: Connecting to ResourceManager at yarnmanager/172.20.0.3:8032
2025-12-06 18:08:21,979 INFO client.DefaultNoHARMFailoverProxyProvider: Connecting to ResourceManager at yarnmanager/172.20.0.3:8032
2025-12-06 18:08:22,129 INFO mapreduce.JobResourceUploader: Disabling Erasure Coding for path: /tmp/hadoop-yarn/staging/root/.staging/job_1765043067955_0006
2025-12-06 18:08:22,449 INFO mapred.FileInputFormat: Total input files to process : 1
2025-12-06 18:08:22,535 INFO mapreduce.JobSubmitter: number of splits:2
2025-12-06 18:08:22,635 INFO mapreduce.JobSubmitter: Submitting tokens for job: job_1765043067955_0006
2025-12-06 18:08:22,635 INFO mapreduce.JobSubmitter: Executing 

In [41]:
!hdfs dfs -cat /pr403/salida5/part-00000

"Mozilla/5.0, 100	


# 3. Análisis temporal de sesión
## Picos de tráfico por horas

In [42]:
%%writefile mapper6.py
#!/usr/bin/env python3

import sys

for line in sys.stdin:
    line = line.strip().split()
    
    fecha = line[3]
    fecha = fecha.strip().split(":")
    hora = fecha[1]
    print(f"{hora}, {1}")

Writing mapper6.py


In [61]:
%%writefile reducer6.py
#!/usr/bin/env python3

import sys

dic = {}

for line in sys.stdin:
    line = line.strip().split(",")
    hora = line[0]
    count = int(line[1])
    if hora in dic:
        dic[hora] = dic[hora] + count
    else:
        dic[hora] = count

for hora, count in dic.items():
    print(f"{hora}, {count}")

Overwriting reducer6.py


In [62]:
!hdfs dfs -rm -r /pr403/salida6

Deleted /pr403/salida6


In [63]:
!hadoop jar \
/usr/local/hadoop/share/hadoop/tools/lib/hadoop-streaming-3.4.0.jar \
-file mapper6.py \
-file reducer6.py \
-mapper mapper6.py \
-reducer reducer6.py \
-input /pr403/logs.log \
-output /pr403/salida6

2025-12-06 18:15:37,885 WARN streaming.StreamJob: -file option is deprecated, please use generic option -files instead.
packageJobJar: [mapper6.py, reducer6.py, /tmp/hadoop-unjar2129959955085644586/] [] /tmp/streamjob1954031514229764434.jar tmpDir=null
2025-12-06 18:15:38,424 INFO client.DefaultNoHARMFailoverProxyProvider: Connecting to ResourceManager at yarnmanager/172.20.0.3:8032
2025-12-06 18:15:38,537 INFO client.DefaultNoHARMFailoverProxyProvider: Connecting to ResourceManager at yarnmanager/172.20.0.3:8032
2025-12-06 18:15:38,684 INFO mapreduce.JobResourceUploader: Disabling Erasure Coding for path: /tmp/hadoop-yarn/staging/root/.staging/job_1765043067955_0009
2025-12-06 18:15:39,034 INFO mapred.FileInputFormat: Total input files to process : 1
2025-12-06 18:15:39,136 INFO mapreduce.JobSubmitter: number of splits:2
2025-12-06 18:15:39,289 INFO mapreduce.JobSubmitter: Submitting tokens for job: job_1765043067955_0009
2025-12-06 18:15:39,290 INFO mapreduce.JobSubmitter: Executing 

In [64]:
!hdfs dfs -cat /pr403/salida6/part-00000

12, 100	


## Tasa de error por endPoint

In [80]:
%%writefile mapper7.py
#!/usr/bin/env python3

import sys

codigo = None

for line in sys.stdin:
    line = line.strip().split()
    url = line[6]
    cod = int(line[8])

    if cod >= 400:
        codigo = "error"
    else:
        codigo = "ok"

    print(f"{url}, {codigo}")

Overwriting mapper7.py


In [106]:
%%writefile reducer7.py
#!/usr/bin/env python3

import sys

total = 0
errores = 0

for line in sys.stdin:
    total = total + 1

    line = line.strip().split(",")
    url = line[0]
    codigo = str(line[1])

    if "error" in codigo:
        errores = errores + 1

print(f"ERRORES TOTALES: {(errores / total) * 100}%")

Overwriting reducer7.py


In [107]:
!hdfs dfs -rm -r /pr403/salida7

Deleted /pr403/salida7


In [108]:
!hadoop jar \
/usr/local/hadoop/share/hadoop/tools/lib/hadoop-streaming-3.4.0.jar \
-file mapper7.py \
-file reducer7.py \
-mapper mapper7.py \
-reducer reducer7.py \
-input /pr403/logs.log \
-output /pr403/salida7

2025-12-06 18:30:19,511 WARN streaming.StreamJob: -file option is deprecated, please use generic option -files instead.
packageJobJar: [mapper7.py, reducer7.py, /tmp/hadoop-unjar255563278939809025/] [] /tmp/streamjob5016497248822100873.jar tmpDir=null
2025-12-06 18:30:20,133 INFO client.DefaultNoHARMFailoverProxyProvider: Connecting to ResourceManager at yarnmanager/172.20.0.3:8032
2025-12-06 18:30:20,240 INFO client.DefaultNoHARMFailoverProxyProvider: Connecting to ResourceManager at yarnmanager/172.20.0.3:8032
2025-12-06 18:30:20,419 INFO mapreduce.JobResourceUploader: Disabling Erasure Coding for path: /tmp/hadoop-yarn/staging/root/.staging/job_1765043067955_0017
2025-12-06 18:30:20,798 INFO mapred.FileInputFormat: Total input files to process : 1
2025-12-06 18:30:20,889 INFO mapreduce.JobSubmitter: number of splits:2
2025-12-06 18:30:20,990 INFO mapreduce.JobSubmitter: Submitting tokens for job: job_1765043067955_0017
2025-12-06 18:30:20,990 INFO mapreduce.JobSubmitter: Executing w

In [109]:
!hdfs dfs -cat /pr403/salida7/part-00000

ERRORES TOTALES: 53.0%	
